In [3]:
# Loading tiny shakespeare 
!curl -O https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1089k  100 1089k    0     0   503k      0  0:00:02  0:00:02 --:--:--  503k  0      0 --:--:-- --:--:-- --:--:--     0


In [1]:
# read it 
with open("input.txt" , 'r' , encoding = 'utf-8') as f:
    words = f.read()
    
len(words)

1115394

In [2]:
# Generating the shakespearean vocabulary

chars = sorted(list(set(words)))
vocab_size = len(chars)

In [3]:
# Creating the stoi and itos mappings

stoi = {s:i for i , s in enumerate(chars)}
itos = {i:s for s, i in stoi.items()}

# encoding and decoding strings 
encode = lambda s : [stoi[ch] for ch in s]
decode = lambda l : ''.join([itos[ch] for ch in l])


encode("Hi Bro!")
decode(encode("Hi Bro!"))

# There is a tradeoff between sequence lenghts and codebook size ;  If codebook size is smaller like in characters
# the sequence lenghts will be very large but if the codebook size is much larger like in GPTs then the sequence 
# length will be much shorter in comparison.

'Hi Bro!'

In [4]:
# Encoding the entire tiny shakespeare dataset into integers and wrap it in a torch tensor

import torch
data = torch.tensor(encode(words) , dtype = torch.long)
print (data.shape , data.dtype)
print (data[:100])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [5]:
# Creating train and cross-val sets

n1 = int(len(data)* 0.9)

train_data = data[:n1]
cv_data = data[:n1]

In [6]:
# We would never train the transformer on the whole dataset together ; instead we will take out random chunks from 
# the dataset and then train on these. 

block_size = 8 # This is the length of the chunk trained on ; can also be called context_length

# Say if we sample the first 9 characters ; these have 8 training examples packed into it

train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [7]:
# Showing the 8 examples as shown before 
x = train_data[:block_size]
y = train_data[1:block_size+1]

for i in range(block_size):
    context = x[:i+1]
    output = y[i]
    print (f'When context is {context} then the output is {output}')
    
# This allows the model to transformer to learn to predict the next word when the given context is 1 char, 2 char, 
# 3 char and so on. Also the transformer cannot recieve more than block_size as the input now and beyond that 
# length if input is given, it will be truncated.

When context is tensor([18]) then the output is 47
When context is tensor([18, 47]) then the output is 56
When context is tensor([18, 47, 56]) then the output is 57
When context is tensor([18, 47, 56, 57]) then the output is 58
When context is tensor([18, 47, 56, 57, 58]) then the output is 1
When context is tensor([18, 47, 56, 57, 58,  1]) then the output is 15
When context is tensor([18, 47, 56, 57, 58,  1, 15]) then the output is 47
When context is tensor([18, 47, 56, 57, 58,  1, 15, 47]) then the output is 58


In [8]:
torch.manual_seed(47)
batch_size = 4 
block_size = 8 

def get_batch(split):
    data = train_data if split == 'train' else cv_data
    ix = torch.randint(len(data) - batch_size , (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1: i+block_size+1] for i in ix])
    return x,y


xb, yb = get_batch('train')

print(xb)
print(yb)

for b in range (batch_size): # batch dimension
    for i in range(block_size): # time dimension
        context = xb[b,:i+1]
        output = yb[b,i]
        print (f'When context is {context} then the output is {output}')

tensor([[63,  1, 57, 54, 43, 43, 42,  1],
        [52,  1, 46, 43,  1, 42, 53, 58],
        [ 1, 51, 39, 49, 43,  1, 39,  1],
        [57,  1, 53, 44,  1, 45, 43, 43]])
tensor([[ 1, 57, 54, 43, 43, 42,  1, 58],
        [ 1, 46, 43,  1, 42, 53, 58, 46],
        [51, 39, 49, 43,  1, 39,  1, 60],
        [ 1, 53, 44,  1, 45, 43, 43, 57]])
When context is tensor([63]) then the output is 1
When context is tensor([63,  1]) then the output is 57
When context is tensor([63,  1, 57]) then the output is 54
When context is tensor([63,  1, 57, 54]) then the output is 43
When context is tensor([63,  1, 57, 54, 43]) then the output is 43
When context is tensor([63,  1, 57, 54, 43, 43]) then the output is 42
When context is tensor([63,  1, 57, 54, 43, 43, 42]) then the output is 1
When context is tensor([63,  1, 57, 54, 43, 43, 42,  1]) then the output is 58
When context is tensor([52]) then the output is 1
When context is tensor([52,  1]) then the output is 46
When context is tensor([52,  1, 46]) th

In [32]:
# We will start off with the simplest model which is Bigram Language Model
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(42)

class BigramLanguageModel(nn.Module):
    
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size , vocab_size)
        
    def forward(self, idx , targets = None):
        
        # idx and targets bring B, T and the embedding table brings the dimension C
        logits = self.token_embedding_table(idx) # B, T, C
            
        if targets is None :
            loss = None 
        else :
            B,T,C = logits.shape  
            # loss = F.cross_entropy(logits, targets)  This won't work because pytorch expects logits in the form B,C,T                 
            loss = F.cross_entropy(logits.view(B*T,C) , targets.view(B*T))    
            # Now channel dimensional is the second dimension as expected and targets becomes B*T instead of B,T 

        return logits , loss  

    def generate(self, idx, max_tokens): # This will sort of do the next word prediction till it generates
    # max_tokens 
    
        for _ in range(max_tokens):
            # idx is B, T
            logits , _ = self(idx)
            # focusing only on the last time step 
            logits = logits[: , -1 , :] # becomes B, C 
            # applyig softmax 
            probs = F.softmax(logits , dim = 1) # also B, C
            # predict the next character in the sequence 
            idx_next = torch.multinomial(probs , num_samples = 1) # B, 1
            # append the predicted the character to the current sequence 
            idx = torch.cat((idx , idx_next), dim = 1) # B, T+1
        return idx 
        
m = BigramLanguageModel(vocab_size)
logits , loss = m(xb , yb) 
loss # To get an good initial guess it should be -log(1/65) ~ 4.17

print (loss)

idx = torch.zeros((1,1) , dtype = torch.long) # Starting with 0 here is a good assumption since it represents the
# next line character in the vocab.
print(decode(m.generate(idx , max_tokens = 100)[0].tolist()))

tensor(4.6156, grad_fn=<NllLossBackward0>)

o$,q&IWqW&xtCjaB?ij&bYRGkF?b; f ,CbwhtERCIfuWr,DzJERjhLlVaF&EjffPHDFcNoGIG'&$qXisWTkJPw
 ,b Xgx?D3sj


In [34]:
# create the learning rate optimizer 
optimizer = torch.optim.AdamW(m.parameters() , lr = 1e-3)

In [43]:
# training the bigram model 

batch_size = 32
for steps in range(10000):
    
    # sample a batch of data 
    xb , yb = get_batch('train')
    
    # evaluate the loss 
    logits , loss = m(xb,yb)
    
    # backprop 
    optimizer.zero_grad(set_to_none = True)
    loss.backward()
    optimizer.step()
    
print(loss.item())

2.383336067199707


In [45]:
print(decode(m.generate(idx , max_tokens = 300)[0].tolist())) # Getting something reasonablish

# Currently the tokens are not talking to each other ; we are only looking at the previous charactr and making the 
# prediction based on that.


ORondorcon y fle hel's t y d ie arthesa wbeve, noife ane doke.
dirend; otichedere, lly my, w t tined
Aspathare tove ge inowhid a ork's ith't med seertho stthive.
ARI t mend gh ot f pak.
LENUKI:
O wh bowischy!
MAnd hrtomistonghemyou e, beathof youn, o, cheng'tof por we y,
OMed ne y, mieyownd t wo ke:


In [46]:
# After this the code has been transferred to a python script and the intuition and thought process for some things
# will be added here 

In [5]:
# Understand the mathematical trick of self-attention

# First we will simply consider attention being taking the average of all the previous and this term ; this is just
# a simple way to start with

# making the demonstration example 
import torch 

torch.manual_seed(43)
B , T , C = 4, 8,2
x = torch.randn(4, 8, 2)
x.shape

torch.Size([4, 8, 2])

In [12]:
# We want x[b,t] = mean (i<=t )(x(b,i))

# Method 1:

xbow = torch.zeros(4, 8, 2)
B,T,C = xbow.shape
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1] # T,C 
        xbow[b,t] = torch.mean(xprev, 0)
print (xprev.shape)
print (xbow.shape)

torch.Size([8, 2])
torch.Size([4, 8, 2])


In [17]:
# Method 2: Matrix multiply allows us to easily do what we want ; tril is just lower triangular matrix 

wei = torch.tril(torch.ones(T,T))
wei = wei/wei.sum(1, keepdim = True)
xbow2 = wei @ x # (B , T , T) @ (B , T , C) = (B , T , C) ; B was braodcasted to the matrix mult takes place across
# T and C

xbow2.shape
torch.allclose(xbow , xbow2)

True

In [26]:
# Method 3: Using Softmax and the final method
from torch.nn import functional as F


tril = torch.tril(torch.ones(T,T))
wei = torch.zeros(T,T) # This here represents the affinity or the interaction strength 
wei = wei.masked_fill(tril == 0 , float('-inf'))
wei = F.softmax(wei , dim = 1)
xbow3 = wei @ x
torch.allclose(xbow2,xbow3)

# We will be using this because it will allow us to first have any weights and then using the tril and the 
# masked_fill what it does basically is restrict it to only attend to the previous time steps and then 
# softmax allows it to sort of take a weighted average since the terms with infinity will go to zero.

True

In [24]:
wei

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])